## 머신러닝을 웹 애플리케이션에서 사용하는 방법
- 프로그램을 실행할 때마다 프로세스를 다시 실행하는 것은 굉장히 비효율적임 (라이브러리를 읽어들이는 시간)
- 머신러닝 프로그램을 서버(프로세스)로 실행해두고 웹 애플리케이션을 통해 머신러닝 학습 결과를 얻는 형태로 프로그램 구현

## 웹 애플리케이션에서 카테고리를 분류하는 모델 사용하기
- 대량의 접근이 없는 경우, 웹 서버 내부에 머신러닝 시스템을 직접 넣는 방법을 사용하여 구현

### 머신러닝 기능을 가진 웹 서버 만들기
- flask 라이브러리 이용하여 API(Application Programming Interface)로 제공
- `tm_server.py`  

```python
import json
import flask
from flask import request
import my_text

# 포트 번호
TM_PORT_NO = 8085
# HTTP 서버 실행하기
app = flask.Flask(__name__)
print('http://localhost:' + str(TM_PORT_NO))

# 루트에 접근할 경우
@app.route('/', methods=['GET'])
def index():
    with open('index.html', 'rb') as f:
        return f.read()

# /api에 접근할 경우
@app.route('/api', methods=['GET'])
def api():
    # URL 매개 변수 추출
    q = request.args.get('q', '')
    if q == '':
        return '{"label":"내용을 입력해주세요", "per":0}'
    print('q=', q)
    # 텍스트 카테고리 판별하기
    label, per, no = my_text.check_genre(q)
    # 결과를 JSON으로 출력
    return json.dumps({
        "label":label,
        "per":per,
        "genre_no":no
    })

if __name__ == '__main__':
    # 서버 실행
    app.run(debug=False, port=TM_PORT_NO, threaded=False)
```

- `python tm_server.py`로 실행
- `http://localhost:8085/api?q=(판정하고 싶은 텍스트)`를 url에 입력
- `index.html`

```html
<DOCTYPE html>
<html>
  <meta charset="utf-8">
  <body>
      <h1>텍스트 카테고리 판별하기</h1>
      <div>
          <textarea id="q" rows="10" cols="60"></textarea>
          <br>
          <button id="qButton">판정</button>
          <div id="result"></div>
      </div>
      <script>
          const qs = (q) => document.querySelector(q)
          window.onload = () => {
              const q = qs('#q')
              const qButton = qs('#qButton')
              const result = qs('#result')
              // 판정 버튼을 눌렀을 때 --- ( ※ 1)
              qButton.onclick = () => {
                  result.innerHTML = "..."
                  // URL 생성하기 --- ( ※ 2)
                  const api = "/api?q=" + encodeURIComponent(q.value)
                  // API에 접근하기 --- ( ※ 3)
                  fetch(api).then((res) => {
                      return res.json() // JSON 응답
                  }).then((data) => {
                      // 결과를 화면에 출력하기 --- ( ※ 4)
                      result.innerHTML =
                          data["label"] +
                          "<span style='font-size:0.5em'>(" + data["per"] + ")</span>"
                  })
              }
          }
      </script>
      <style>
          #result {
              padding: 10px;
              font-size: 2em;
              color: red;
          }
          
          #q {
              background-color: #fffff0;
          }
      </style>
  </body>
</html>
```

- `http://localhost:8085` 접속

## 참고

### 웹 서비스와 조합해서 사용할 때의 머신러닝 시스템 유지 보수
- 어려운 점 : 한 번 만들었다고 끝이 아님
- 따라서 정기적으로 데이터를 다시 분석하고 학습시켜야 함
- (웹 서비스) 데이터 저장 > (데이터 베이스) 교사 데이터 업데이트 > (머신러닝 시스템) 추천 등의 활용 > (웹 서비스)
- 이러한 흐름을 자동화하는 것이 좋으며, 정답률이 급격하게 떨어지는 경우 개발자에게 알림을 보내는 시스템이 있으면 어느정도 대응 가능